# Install Dependencies
Install required packages including diffusers, transformers, torch, and necessary dependencies. Include environment-specific installations for Colab vs local setup.

In [ ]:
# Install Dependencies

# Check if the notebook is running in Google Colab
!pip3 install diffusers transformers
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

# Setup Environment Check
Check if running in Colab or local environment, set device (CPU/GPU/MPS), and configure appropriate paths.

In [ ]:
# Setup Environment Check

import torch
import sys
import os

# Check if the notebook is running in Google Colab
if 'google.colab' in sys.modules:
    # Set device to GPU if available
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print("Running on Google Colab")
else:
    # Set device to GPU if available, otherwise CPU
    if torch.cuda.is_available():
        device = torch.device('cuda')
    elif torch.backends.mps.is_available():
        device = torch.device('mps')
    else:
        device = torch.device('cpu')
    print("Running on local environment")

print(f"Using device: {device}")

model_dir = "models"
checkpoint_dir = os.path.join(model_dir, "checkpoint")
lora_dir = os.path.join(model_dir, "lora")

# Download and Load SDXL Model
Download and load the base SDXL model using diffusers library, with proper error handling and memory optimization.

In [ ]:
# Download Models

default_models = {
    "dynavision-xl": "https://civitai.com/api/download/models/297740?type=Model&format=SafeTensor&size=pruned&fp=fp16",
    "realvis-xl": "https://civitai.com/api/download/models/798204?type=Model&format=SafeTensor&size=full&fp=fp16"
}

for model_name, model_url in default_models.items():
    print(f"Downloading {model_name} model")
    !aria2c --auto-file-renaming=false --continue --dir=models/checkpoint/ $model_url

In [ ]:

from diffusers import StableDiffusionPipeline, AutoPipelineForText2Image

# Define the model ID for the SDXL model
# @markdown ### **Model Config**
model_id = "stable-diffusion-v1.5" #@param {type:"string"}

# List all models in the model directory
models = os.listdir(checkpoint_dir)
for model in models:
    print(model)

# try:
#     # Load the SDXL model
#     AutoPipelineForText2Image.from_pretrained()
#     pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
#     pipe = pipe.to(device)
#     print("Model loaded successfully")
# except Exception as e:
#     print(f"Error loading model: {e}")

# Load LoRA Models
Function to load and apply LoRA models to the base SDXL model, with support for multiple LoRA weights.

In [ ]:
# Load LoRA Models

from transformers import AutoModelForCausalLM, AutoTokenizer

def load_lora_models(base_model, lora_weights_list):
    """
    Load and apply LoRA models to the base SDXL model.
    
    Args:
    - base_model: The base SDXL model.
    - lora_weights_list: List of paths or identifiers for LoRA weights.
    
    Returns:
    - The base model with LoRA weights applied.
    """
    for lora_weights in lora_weights_list:
        try:
            # Load the LoRA model
            lora_model = AutoModelForCausalLM.from_pretrained(lora_weights)
            # Apply LoRA weights to the base model
            base_model = lora_model.apply_lora(base_model)
            print(f"LoRA weights from {lora_weights} applied successfully")
        except Exception as e:
            print(f"Error loading LoRA weights from {lora_weights}: {e}")
    return base_model

# Example usage
lora_weights_list = ["path/to/lora/weights1", "path/to/lora/weights2"]
pipe = load_lora_models(pipe, lora_weights_list)

# Define Image Generation Function
Create a function that handles prompt processing, inference pipeline setup, and image generation with customizable parameters.

In [ ]:
# Define Image Generation Function

def generate_image(prompt, num_inference_steps=50, guidance_scale=7.5, height=512, width=512):
    """
    Generate an image from a text prompt using the SDXL model with LoRA models.
    
    Args:
    - prompt: The text prompt for image generation.
    - num_inference_steps: Number of inference steps for the generation process.
    - guidance_scale: Guidance scale for classifier-free guidance.
    - height: Height of the generated image.
    - width: Width of the generated image.
    
    Returns:
    - Generated image.
    """
    try:
        # Generate image
        image = pipe(prompt, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale, height=height, width=width).images[0]
        return image
    except Exception as e:
        print(f"Error generating image: {e}")
        return None

# Example usage
prompt = "A futuristic cityscape at sunset"
generated_image = generate_image(prompt)
if generated_image:
    generated_image.show()

# Generate Images with Different Prompts
Demonstrate image generation with various prompts and LoRA combinations, including parameter adjustments.

In [ ]:
# Generate Images with Different Prompts

# Define a list of prompts
prompts = [
    "A futuristic cityscape at sunset",
    "A serene mountain landscape with a river",
    "A bustling market in a medieval town",
    "A spaceship flying through a nebula",
    "A fantasy castle surrounded by dragons"
]

# Define a list of LoRA weight combinations (for demonstration purposes, using the same weights)
lora_combinations = [
    ["path/to/lora/weights1"],
    ["path/to/lora/weights2"],
    ["path/to/lora/weights1", "path/to/lora/weights2"]
]

# Generate and display images for each prompt and LoRA combination
for prompt in prompts:
    for lora_weights in lora_combinations:
        # Load the base model with the current LoRA weights
        pipe_with_lora = load_lora_models(pipe, lora_weights)
        
        # Generate the image
        generated_image = generate_image(prompt)
        
        # Display the image
        if generated_image:
            print(f"Prompt: {prompt} | LoRA Weights: {lora_weights}")
            generated_image.show()

# Save Generated Images
Save generated images with metadata, including the prompt and parameters used for generation.

In [ ]:
# Save Generated Images

import os
from PIL import Image
import json

def save_image_with_metadata(image, prompt, parameters, save_dir="generated_images"):
    """
    Save the generated image along with its metadata.
    
    Args:
    - image: The generated image to save.
    - prompt: The text prompt used for image generation.
    - parameters: Dictionary of parameters used for generation.
    - save_dir: Directory to save the images and metadata.
    """
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    # Define the image filename
    image_filename = os.path.join(save_dir, f"{prompt[:50].replace(' ', '_')}.png")
    
    # Save the image
    image.save(image_filename)
    
    # Define the metadata filename
    metadata_filename = os.path.join(save_dir, f"{prompt[:50].replace(' ', '_')}.json")
    
    # Save the metadata
    with open(metadata_filename, 'w') as f:
        json.dump({
            "prompt": prompt,
            "parameters": parameters
        }, f, indent=4)
    
    print(f"Image and metadata saved: {image_filename}, {metadata_filename}")

# Example usage
if generated_image:
    parameters = {
        "num_inference_steps": 50,
        "guidance_scale": 7.5,
        "height": 512,
        "width": 512
    }
    save_image_with_metadata(generated_image, prompt, parameters)